In [1]:
import pandas as pd

In [3]:
proc = pd.read_csv("data/procedures_preprocess_threshold_nonfiltered.csv")

In [9]:
#diag = pd.read_csv("diagnosis_preprocess.csv")
diag = pd.read_csv("data/diagnosis_preprocess_nonfiltered.csv")
#"data/diagnosis_preprocess_nonfiltered.csv"
#drugs = pd.read_csv("drugs_preprosss.csv")
drugs = pd.read_csv("data/drugs1_preprosss_non_preprocess.csv")
drugs1 = pd.read_csv("data/drugs2_preprosss_non_preprocess.csv")
drugs.head()

,Unnamed: 0,SUBJECT_ID,HADM_ID,STARTDATE,DRUG,NDC,RXCUI,ATC4,ATC3
0,0,2,163353,2138-07-18,Ampicillin Sodium,63323038810,1721474,J01CA,J01C
1,1,8,159514,2117-11-20,NEO*IV*Ampicillin Sodium,63323038810,1721474,J01CA,J01C
2,2,10,184167,2103-06-28,NEO*IV*Ampicillin Sodium,63323038810,1721474,J01CA,J01C
3,3,27,134931,2191-12-01,NEO*IV*Ampicillin Sodium,63323038810,1721474,J01CA,J01C
4,4,50,132761,2112-06-24,Ampicillin Sodium,63323038810,1721474,J01CA,J01C


In [10]:
drugs.shape

(2261453, 9)

In [11]:
drugs1.shape

(1756871, 8)

In [21]:
adm = pd.read_csv("/Users/cgarciay/Desktop/Laval_Master_Computer/research/MIMIC/ADMISSIONS.csv.gz")


In [22]:
adm.shape

(58976, 19)

# Adm

In [23]:
import polars as pl
#df = pl.read_parquet("df_filtered.parquet")
df =pl.read_parquet("/Users/cgarciay/Desktop/Laval_Master_Computer/research/Synthetic-Data-Deep-Learning copy/df_non_filtered.parquet")
adm = df[[
 'ADMISSION_TYPE',
 'ADMISSION_LOCATION',
 'DISCHARGE_LOCATION',
 'INSURANCE',
 'LANGUAGE',
 'RELIGION',
 'MARITAL_STATUS',
 'ETHNICITY',
 'DIAGNOSIS',
 'HOSPITAL_EXPIRE_FLAG',

 'GENDER',
 'EXPIRE_FLAG',

  'DOB','SUBJECT_ID',
 'HADM_ID','ADMITTIME','DISCHTIME'

]].to_pandas()

adm["DISCHTIME"] = pd.to_datetime(adm["DISCHTIME"])
adm["ADMITTIME"] = pd.to_datetime(adm["ADMITTIME"])
adm["LOSRD"] = adm["DISCHTIME"] - adm["ADMITTIME"]

In [20]:
df.shape

(58976, 60)

In [14]:
adm["DISCHTIME"] = pd.to_datetime(adm["DISCHTIME"])
adm["ADMITTIME"] = pd.to_datetime(adm["ADMITTIME"])
adm["LOSRD"] = adm["DISCHTIME"] - adm["ADMITTIME"]

In [109]:
''''''adm = pd.read_csv("dataset/ADMISSIONS.csv.gz")
desired_ids = pd.read_csv("ids.csv")["Subjects_id"]
condition = adm['SUBJECT_ID'].isin(desired_ids)

# Apply the condition to filter the DataFrame
adm = adm[condition]
''''''

In [24]:
adm['LOSRD']  = [i.days for i in adm['LOSRD']]

In [25]:
adm['DOB'] = pd.to_datetime(adm['DOB'])
adm['ADMITTIME'] = pd.to_datetime(adm['ADMITTIME'])
adm['ADMITTIME'] = [timestamp.to_pydatetime() for timestamp in adm['ADMITTIME']]
adm['DOB'] = [timestamp.to_pydatetime() for timestamp in adm['DOB']]
adm['ADMITTIME'] = [timestamp.date() for timestamp in adm['ADMITTIME']]
adm['DOB'] = [timestamp.date() for timestamp in adm['DOB']]
adm["age"] = (adm['ADMITTIME'].to_numpy() - adm['DOB'].to_numpy())
adm["year_age"] = [i.days/365 for i in adm["age"]]

Replace does person that have more than 90 years.

In [26]:
def replace_age(age):
    if age > 100:
        return 90
    else:
        return age

# Apply the custom function to the 'Age' column
adm['year_age'] = adm['year_age'].apply(replace_age)


In [27]:
# Group by 'SUBJECT_ID' and 'HADM_ID' and aggregate all columns into lists
grouped = adm.groupby(['SUBJECT_ID', 'HADM_ID']).agg(lambda x: x.tolist())

# Reset the index to make 'SUBJECT_ID' and 'HADM_ID' regular columns
grouped_adm = grouped.reset_index()
grouped_adm

,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,GENDER,EXPIRE_FLAG,DOB,ADMITTIME,DISCHTIME,LOSRD,age,year_age
0,10,184167,[NEWBORN],[PHYS REFERRAL/NORMAL DELI],[SHORT TERM HOSPITAL],[Medicaid],[None],[UNOBTAINABLE],[None],[BLACK/AFRICAN AMERICAN],[NEWBORN],[0],[F],[0],[2103-06-28],[2103-06-28],[2103-07-06 12:10:00],[8],[0:00:00],[0.0]
1,100,153952,[ELECTIVE],[PHYS REFERRAL/NORMAL DELI],[HOME],[Private],[None],[CATHOLIC],[WIDOWED],[UNKNOWN/NOT SPECIFIED],[AORTIC INSUFFICIENCY\AORTIC VALVE REPLACEMENT...,[0],[F],[0],[2085-08-31],[2157-08-10],[2157-08-18 19:54:00],[8],"[26276 days, 0:00:00]",[71.98904109589041]
2,1000,143040,[URGENT],[TRANSFER FROM HOSP/EXTRAM],[DEAD/EXPIRED],[Private],[None],[UNOBTAINABLE],[MARRIED],[UNKNOWN/NOT SPECIFIED],[HEART FAILURE],[1],[M],[1],[2074-05-06],[2144-01-19],[2144-02-25 06:05:00],[36],"[25459 days, 0:00:00]",[69.75068493150685]
3,10000,187813,[EMERGENCY],[TRANSFER FROM HOSP/EXTRAM],[SHORT TERM HOSPITAL],[Self Pay],[SPAN],[CATHOLIC],[None],[HISPANIC OR LATINO],[END STAGE LIVER DISEASE;END STAGE KIDNEY DISE...,[0],[M],[0],[2136-12-12],[2186-08-10],[2186-08-31 17:00:00],[20],"[18138 days, 0:00:00]",[49.69315068493151]
4,10001,118420,[NEWBORN],[PHYS REFERRAL/NORMAL DELI],[HOME],[Government],[None],[BUDDHIST],[None],[ASIAN],[NEWBORN],[0],[M],[0],[2107-03-18],[2107-03-18],[2107-03-20 11:59:00],[1],[0:00:00],[0.0]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,9999,117856,[EMERGENCY],[TRANSFER FROM HOSP/EXTRAM],[DEAD/EXPIRED],[Medicaid],[None],[CATHOLIC],[SINGLE],[UNKNOWN/NOT SPECIFIED],[LIVER FAILURE],[1],[M],[1],[2091-02-27],[2134-12-26],[2134-12-28 04:45:00],[1],"[16007 days, 0:00:00]",[43.85479452054795]
58972,99991,151118,[ELECTIVE],[PHYS REFERRAL/NORMAL DELI],[HOME],[Private],[ENGL],[NOT SPECIFIED],[MARRIED],[WHITE],[DIVERTICULITIS/SDA],[0],[M],[0],[2137-04-07],[2184-12-24],[2185-01-05 12:15:00],[12],"[17428 days, 0:00:00]",[47.74794520547945]
58973,99992,197084,[EMERGENCY],[CLINIC REFERRAL/PREMATURE],[SNF],[Medicare],[ENGL],[CATHOLIC],[WIDOWED],[WHITE],[RETROPERITONEAL HEMORRHAGE],[0],[F],[0],[2078-10-17],[2144-07-25],[2144-07-28 17:56:00],[2],"[24022 days, 0:00:00]",[65.81369863013698]
58974,99995,137810,[ELECTIVE],[PHYS REFERRAL/NORMAL DELI],[HOME],[Medicare],[ENGL],[NOT SPECIFIED],[MARRIED],[WHITE],[ABDOMINAL AORTIC ANEURYSM/SDA],[0],[F],[1],[2058-05-29],[2147-02-08],[2147-02-11 13:15:00],[3],"[32396 days, 0:00:00]",[88.75616438356164]


In [28]:
grouped_adm.HADM_ID.nunique()

58976

In [29]:
grouped_adm.SUBJECT_ID.nunique()

46520

# Procedure

In [30]:
proc.rename(columns={ "'CCS CATEGORY DESCRIPTION'":'CCS CATEGORY DESCRIPTION_proc',
        'CCS CODES':'CCS CODES_proc','cat_threshold .95 most frequent': 'cat_threshold .95 most frequent_proc',"ICD9_CODE":"ICD9_CODE_procedures"},inplace = True)

In [31]:
proc = proc[['SUBJECT_ID', 'HADM_ID', "ICD9_CODE_procedures",  'CCS CATEGORY DESCRIPTION_proc',
        'CCS CODES_proc',
       'cat_threshold .95 most frequent_proc']]

In [32]:

# Group by 'SUBJECT_ID' and 'HADM_ID' and aggregate all columns into lists
grouped = proc.groupby(['SUBJECT_ID', 'HADM_ID']).agg(lambda x: x.tolist())

# Reset the index to make 'SUBJECT_ID' and 'HADM_ID' regular columns
grouped_proc = grouped.reset_index()
grouped_proc

,SUBJECT_ID,HADM_ID,ICD9_CODE_procedures,CCS CATEGORY DESCRIPTION_proc,CCS CODES_proc,cat_threshold .95 most frequent_proc
0,2,163353,[9955],['Vaccinations '],['228'],[9955]
1,3,145834,"[3893, 9672, 9604, 966, 9962, 8964]","['Ot vasc cath ', 'Mech ventil ', 'Me...","['54', '216', '216', '223', '225', '204']","[3893, 9672, 9604, 966, 9962, 8964]"
2,4,185777,"[3893, 8872, 3323]","['Ot vasc cath ', 'Echocardiogr ', 'Br...","['54', '193', '37']","[3893, 8872, 3323]"
3,5,178980,[9955],['Vaccinations '],['228'],[9955]
4,6,107064,"[5569, 9904, 3957, 3806, 91]","['Kidney trans ', 'Blood transf ', 'Ot...","['105', '222', '61', '61', '231']","[5569, 9904, 3957, -1, -1]"
...,...,...,...,...,...,...
52238,99985,176670,"[3893, 9672, 9604, 966, 14]","['Ot vasc cath ', 'Mech ventil ', 'Me...","['54', '216', '216', '223', '231']","[3893, 9672, 9604, 966, 14]"
52239,99991,151118,"[5491, 544, 5459, 5412, 4823, 5425, 1736]","['Abd paracent ', 'OR GI Rx prs ', 'Ex...","['88', '99', '90', '99', '77', '97', '78']","[5491, 544, 5459, 5412, 4823, 5425, -1]"
52240,99992,197084,"[8847, 3979]","['Ot arterio ', 'Ot vessels ']","['191', '61']","[8847, 3979]"
52241,99995,137810,"[3971, 8339, 1771]","['Aort resect ', 'Rx prs musc ', 'Ot...","['52', '160', '61']","[3971, 8339, -1]"


In [33]:
grouped_proc.HADM_ID.nunique()

52243

In [34]:
grouped_proc.SUBJECT_ID.nunique()

42214

# Diagnosis

In [44]:
diag.columns

Index(['Unnamed: 0', 'HADM_ID', 'SUBJECT_ID', 'ICD9_CODE_diagnosis',
       'CCS_CODES_diagnosis', 'CCS_description_diagnosis', 'LEVE3 CODES',
       'threshold_0.95_diagnosis', 'threshold_0.88_diagnosis',
       'threshold_0.98_diagnosis', 'threshold_0.999_diagnosis'],
      dtype='object')

In [43]:
diag.rename(columns={ "ICD9_CODE":"ICD9_CODE_diagnosis","CCS CODES": "CCS_CODES_diagnosis","'CCS CATEGORY DESCRIPTION'":"CCS_description_diagnosis"
        
                     
         , "threshold_0.95":"threshold_0.95_diagnosis","threshold_0.88":"threshold_0.88_diagnosis" 
          
         ,"threshold_0.98":"threshold_0.98_diagnosis","threshold_0.999":"threshold_0.999_diagnosis"} ,inplace = True)

In [ ]:
'''diag = diag[[ 'SUBJECT_ID', 'HADM_ID',  'CCS CODES',
       'LEVE3 CODES',  'icd9_category', 'cat_threshold .75',"'CCS CATEGORY DESCRIPTION'" ,"ICD9_CODE_diagnosis"]]'''

In [45]:
diag = diag[['HADM_ID', 'SUBJECT_ID', 'ICD9_CODE_diagnosis',
       'CCS_CODES_diagnosis', 'CCS_description_diagnosis', 'LEVE3 CODES',
       'threshold_0.95_diagnosis', 'threshold_0.88_diagnosis',
       'threshold_0.98_diagnosis', 'threshold_0.999_diagnosis']]

In [46]:
diag.head()

,HADM_ID,SUBJECT_ID,ICD9_CODE_diagnosis,CCS_CODES_diagnosis,CCS_description_diagnosis,LEVE3 CODES,threshold_0.95_diagnosis,threshold_0.88_diagnosis,threshold_0.98_diagnosis,threshold_0.999_diagnosis
0,172335,109,40301,99,Htn complicn,403,40301,40301,40301,40301
1,172335,109,486,122,Pneumonia,486,486,486,486,486
2,172335,109,58281,156,Nephritis,582,58281,-1,58281,58281
3,172335,109,5855,158,Chr kidney disease,585,5855,-1,5855,5855
4,172335,109,4254,97,Carditis,425,4254,4254,4254,4254


In [47]:

# Group by 'SUBJECT_ID' and 'HADM_ID' and aggregate all columns into lists
grouped = diag.groupby(['SUBJECT_ID', 'HADM_ID']).agg(lambda x: x.tolist())

# Reset the index to make 'SUBJECT_ID' and 'HADM_ID' regular columns
grouped_diag = grouped.reset_index()
grouped_diag

,SUBJECT_ID,HADM_ID,ICD9_CODE_diagnosis,CCS_CODES_diagnosis,CCS_description_diagnosis,LEVE3 CODES,threshold_0.95_diagnosis,threshold_0.88_diagnosis,threshold_0.98_diagnosis,threshold_0.999_diagnosis
0,2,163353,"[V3001, V053, V290]","[218, 10, 256]","[Liveborn, Immuniz/scrn, Exam/eval]","[V30, V05, V29]","[V3001, V053, V290]","[V3001, V053, V290]","[V3001, V053, V290]","[V3001, V053, V290]"
1,3,145834,"[0389, 78559, 5849, 4275, 41071, 4280, 6826, 4...","[2, 249, 157, 107, 100, 108, 197, 97, 52]","[Septicemia, Shock, Ac renl fail, Cardia arrst...","[038, 785, 584, 427, 410, 428, 682, 425, 263]","[0389, 78559, 5849, 4275, 41071, 4280, 6826, 4...","[0389, 78559, 5849, 4275, 41071, 4280, 6826, 4...","[0389, 78559, 5849, 4275, 41071, 4280, 6826, 4...","[0389, 78559, 5849, 4275, 41071, 4280, 6826, 4..."
2,4,185777,"[042, 1363, 7994, 2763, 7907, 5715, 04111, V09...","[5, 122, 52, 55, 2, 151, 3, 3, 2617]","[HIV infectn, Pneumonia, Nutrit defic, Fluid/e...","[042, 136, 799, 276, 790, 571, 041, V09, E93]","[042, 1363, 7994, 2763, 7907, 5715, 04111, V09...","[042, -1, 7994, 2763, 7907, 5715, 04111, V090,...","[042, 1363, 7994, 2763, 7907, 5715, 04111, V09...","[042, 1363, 7994, 2763, 7907, 5715, 04111, V09..."
3,5,178980,"[V3000, V053, V290]","[218, 10, 256]","[Liveborn, Immuniz/scrn, Exam/eval]","[V30, V05, V29]","[V3000, V053, V290]","[V3000, V053, V290]","[V3000, V053, V290]","[V3000, V053, V290]"
4,6,107064,"[40391, 4440, 9972, 2766, 2767, 2859, 2753, V1...","[99, 116, 238, 55, 55, 59, 58, 663]","[Htn complicn, Art embolism, Complic proc, Flu...","[403, 444, 997, 276, 276, 285, 275, V15]","[40391, 4440, 9972, 2766, 2767, 2859, 2753, V1...","[40391, -1, 9972, 2766, 2767, 2859, 2753, V1582]","[40391, 4440, 9972, 2766, 2767, 2859, 2753, V1...","[40391, 4440, 9972, 2766, 2767, 2859, 2753, V1..."
...,...,...,...,...,...,...,...,...,...,...
58924,99985,176670,"[0389, 51881, 48241, 4870, 78552, V4281, 99592...","[2, 131, 122, 123, 249, 259, 2, 48, 53, 59, 13...","[Septicemia, Adlt resp fl, Pneumonia, Influenz...","[038, 518, 482, 487, 785, V42, 995, 244, 272, ...","[0389, 51881, 48241, 4870, 78552, V4281, 99592...","[0389, 51881, 48241, -1, 78552, -1, 99592, 244...","[0389, 51881, 48241, 4870, 78552, V4281, 99592...","[0389, 51881, 48241, 4870, 78552, V4281, 99592..."
58925,99991,151118,"[56211, 0389, 5570, 5849, 99592, 56081, 78959,...","[146, 2, 114, 157, 2, 145, 151, 143, 163, 99, ...","[Diverticulos, Septicemia, Perip athero, Ac re...","[562, 038, 557, 584, 995, 560, 789, 553, 788, ...","[56211, 0389, 5570, 5849, 99592, 56081, 78959,...","[56211, 0389, 5570, 5849, 99592, 56081, 78959,...","[56211, 0389, 5570, 5849, 99592, 56081, 78959,...","[56211, 0389, 5570, 5849, 99592, 56081, 78959,..."
58926,99992,197084,"[9999, 56881, 5772, 2851, 5849, 5799, 72992, 5...","[238, 155, 152, 60, 157, 155, 211, 138, 98, 53...","[Complic proc, Other GI dx, Pancreas dx, Acut ...","[999, 568, 577, 285, 584, 579, 729, 530, 401, ...","[9999, 56881, 5772, 2851, 5849, -1, 72992, 530...","[9999, 56881, 5772, 2851, 5849, -1, -1, 53081,...","[9999, 56881, 5772, 2851, 5849, 5799, 72992, 5...","[9999, 56881, 5772, 2851, 5849, 5799, 72992, 5..."
58927,99995,137810,"[4414, 42833, 99812, 2851, 4241, 25000, 99811,...","[115, 108, 238, 60, 96, 49, 238, 237, 2616, 53...","[Aneurysm, chf;nonhp, Complic proc, Acut p-h a...","[441, 428, 998, 285, 424, 250, 998, 996, E87, ...","[4414, 42833, 99812, 2851, 4241, 25000, 99811,...","[4414, 42833, 99812, 2851, 4241, 25000, 99811,...","[4414, 42833, 99812, 2851, 4241, 25000, 99811,...","[4414, 42833, 99812, 2851, 4241, 25000, 99811,..."


In [49]:
grouped_diag.SUBJECT_ID.nunique()

46517

In [50]:
grouped_diag.HADM_ID.nunique()

58929

# Drugs

In [51]:
drugs.columns

Index(['Unnamed: 0', 'SUBJECT_ID', 'HADM_ID', 'STARTDATE', 'DRUG', 'NDC',
       'RXCUI', 'ATC4', 'ATC3'],
      dtype='object')

In [52]:
drugs1.columns

Index(['Unnamed: 0', 'HADM_ID', 'SUBJECT_ID', 'DRUG', 'threshold_0.88',
       'threshold_0.95', 'threshold_0.98', 'threshold_0.999'],
      dtype='object')

In [53]:
drugs = drugs[[ 'SUBJECT_ID', 'HADM_ID' , 'DRUG', 'ATC4', 'ATC3']]
drugs1 = drugs1[[ 'HADM_ID', 'SUBJECT_ID', 'DRUG', 'threshold_0.88',
       'threshold_0.95', 'threshold_0.98', 'threshold_0.999']]

In [54]:

# Group by 'SUBJECT_ID' and 'HADM_ID' and aggregate all columns into lists
grouped = drugs.groupby(['SUBJECT_ID', 'HADM_ID']).agg(lambda x: x.tolist())

# Reset the index to make 'SUBJECT_ID' and 'HADM_ID' regular columns
grouped_drugs = grouped.reset_index()
grouped_drugs.shape

(49929, 5)

In [56]:
# Group by 'SUBJECT_ID' and 'HADM_ID' and aggregate all columns into lists
grouped1 = drugs1.groupby(['SUBJECT_ID', 'HADM_ID']).agg(lambda x: x.tolist())

# Reset the index to make 'SUBJECT_ID' and 'HADM_ID' regular columns
grouped_drugs1 = grouped1.reset_index()
grouped_drugs1.shape

(50216, 7)

In [59]:
print(grouped_drugs.SUBJECT_ID.nunique())
print(grouped_drugs.HADM_ID.nunique())


39087
49929


In [60]:
print(grouped_drugs1.SUBJECT_ID.nunique())
print(grouped_drugs1.HADM_ID.nunique())

39363
50216


49929

In [33]:
adm.shape

(19993, 23)

In [34]:
proc.shape

(81755, 6)

In [61]:
adm['SUBJECT_ID'] = adm['SUBJECT_ID'].astype(int)
adm['HADM_ID'] = adm['HADM_ID'].astype(int)

In [62]:
merge_proc = pd.merge(adm, grouped_proc, on=['SUBJECT_ID', 'HADM_ID'], how='left')
merge_proc

,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,...,HADM_ID,ADMITTIME,DISCHTIME,LOSRD,age,year_age,ICD9_CODE_procedures,CCS CATEGORY DESCRIPTION_proc,CCS CODES_proc,cat_threshold .95 most frequent_proc
0,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,None,UNOBTAINABLE,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,0,...,165315,2196-04-09,2196-04-10 15:54:00,1,"23714 days, 0:00:00",64.969863,"[9671, 9604, 9634]","['Mech ventil ', 'Mech ventil ', 'nO...","['216', '216', '93']","[9671, 9604, 9634]"
1,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,None,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,...,152223,2153-09-03,2153-09-08 19:10:00,5,"25980 days, 0:00:00",71.178082,"[3613, 3615, 3961, 8872, 9904, 9907, 9905]","['CABG ', 'CABG ', 'Ex...","['44', '44', '50', '193', '222', '222', '222']","[3613, 3615, 3961, 8872, 9904, 9907, 9905]"
2,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,BRAIN MASS,0,...,124321,2157-10-18,2157-10-25 14:00:00,6,"27486 days, 0:00:00",75.304110,[151],['Inc/exc CNS '],['1'],[151]
3,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,None,PROTESTANT QUAKER,SINGLE,WHITE,INTERIOR MYOCARDIAL INFARCTION,0,...,161859,2139-06-06,2139-06-09 12:48:00,2,"14250 days, 0:00:00",39.041096,"[3601, 3723, 8856, 9920, 8852, 3607]","['PTCA ', 'Cardiac cath ', 'Ca...","['45', '47', '47', '231', '47', '63']","[3601, 3723, 8856, 9920, 8852, 3607]"
4,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,None,UNOBTAINABLE,MARRIED,WHITE,ACUTE CORONARY SYNDROME,0,...,129635,2160-11-02,2160-11-05 14:55:00,3,"21531 days, 0:00:00",58.989041,"[3601, 8856, 8856, 8853, 9920, 9920, 3722, 372...","['PTCA ', 'Cardiac cath ', 'Ca...","['45', '47', '47', '47', '231', '231', '47', '...","[3601, 8856, 8856, 8853, 9920, 9920, 3722, 372..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,NOT SPECIFIED,SINGLE,WHITE,TRAUMA,0,...,191113,2131-03-30,2131-04-02 15:02:00,2,"7085 days, 0:00:00",19.410959,"[9671, 2751, 881, 8659]","['Mech ventil ', 'nOR nose mou ', 'Ot...","['216', '32', '19', '171']","[9671, 2751, 881, 8659]"
58972,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,SAH,1,...,101071,2151-03-05,2151-03-06 09:10:00,0,"30480 days, 0:00:00",83.506849,[9671],['Mech ventil '],['216'],[9671]
58973,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,RENAL CANCER/SDA,0,...,122631,2200-09-12,2200-09-20 12:08:00,8,"15534 days, 0:00:00",42.558904,"[4719, 5551, 4576, 4594, 1742]","['Appendectomy ', 'Nephrectomy ', 'Co...","['80', '104', '78', '96', '231']","[4719, 5551, 4576, 4594, -1]"
58974,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,ENGL,CATHOLIC,MARRIED,WHITE,S/P FALL,0,...,170407,2128-11-11,2128-12-22 13:11:00,41,"22195 days, 0:00:00",60.808219,"[9671, 3893, 9604, 966]","['Mech ventil ', 'Ot vasc cath ', 'Me...","['216', '54', '216', '223']","[9671, 3893, 9604, 966]"


In [63]:
print(grouped_adm.shape)
print(merge_proc.SUBJECT_ID.nunique())
merge_proc.HADM_ID.nunique()

(58976, 20)
46520


58976

In [64]:
merge_diag = pd.merge(merge_proc, grouped_diag, on=['SUBJECT_ID', 'HADM_ID'], how='left')
merge_diag.head()

,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,...,CCS CODES_proc,cat_threshold .95 most frequent_proc,ICD9_CODE_diagnosis,CCS_CODES_diagnosis,CCS_description_diagnosis,LEVE3 CODES,threshold_0.95_diagnosis,threshold_0.88_diagnosis,threshold_0.98_diagnosis,threshold_0.999_diagnosis
0,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,None,UNOBTAINABLE,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,0,...,"['216', '216', '93']","[9671, 9604, 9634]","[9678, 9693, E9502, E9503, 3488, 29620, 4019]","[242, 241, 662, 662, 95, 657, 98]","[Poisn ot med, Poison psych, Suicide and inten...","[967, 969, E95, E95, 348, 296, 401]","[-1, 9693, -1, E9503, 3488, 29620, 4019]","[-1, -1, -1, E9503, 3488, 29620, 4019]","[9678, 9693, E9502, E9503, 3488, 29620, 4019]","[9678, 9693, E9502, E9503, 3488, 29620, 4019]"
1,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,None,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,...,"['44', '44', '50', '193', '222', '222', '222']","[3613, 3615, 3961, 8872, 9904, 9907, 9905]","[41401, 4111, 4241, V4582, 2724, 4019, 60000, ...","[101, 101, 96, 101, 53, 98, 164, 94]","[Coron athero, Coron athero, Hrt valve dx, Cor...","[414, 411, 424, V45, 272, 401, 600, 389]","[41401, 4111, 4241, V4582, 2724, 4019, 60000, ...","[41401, 4111, 4241, V4582, 2724, 4019, 60000, ...","[41401, 4111, 4241, V4582, 2724, 4019, 60000, ...","[41401, 4111, 4241, V4582, 2724, 4019, 60000, ..."
2,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,BRAIN MASS,0,...,['1'],[151],"[2252, 3485, 78039, 4241, 4019, 2720, 2724, V4...","[47, 95, 83, 96, 98, 53, 53, 101, 259, 663]","[Ot bnign neo, Oth nerv dx, Epilepsy/cnv, Hrt ...","[225, 348, 780, 424, 401, 272, 272, V45, V45, ...","[2252, 3485, 78039, 4241, 4019, 2720, 2724, V4...","[2252, 3485, 78039, 4241, 4019, 2720, 2724, V4...","[2252, 3485, 78039, 4241, 4019, 2720, 2724, V4...","[2252, 3485, 78039, 4241, 4019, 2720, 2724, V4..."
3,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,None,PROTESTANT QUAKER,SINGLE,WHITE,INTERIOR MYOCARDIAL INFARCTION,0,...,"['45', '47', '47', '231', '47', '63']","[3601, 3723, 8856, 9920, 8852, 3607]","[41041, 41401, 53081, 25000]","[100, 101, 138, 49]","[Acute MI, Coron athero, Esophgeal dx, DiabMel...","[410, 414, 530, 250]","[41041, 41401, 53081, 25000]","[41041, 41401, 53081, 25000]","[41041, 41401, 53081, 25000]","[41041, 41401, 53081, 25000]"
4,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,None,UNOBTAINABLE,MARRIED,WHITE,ACUTE CORONARY SYNDROME,0,...,"['45', '47', '47', '47', '231', '231', '47', '...","[3601, 8856, 8856, 8853, 9920, 9920, 3722, 372...","[41071, 25011, 41401, 4019]","[100, 50, 101, 98]","[Acute MI, DiabMel w/cm, Coron athero, HTN]","[410, 250, 414, 401]","[41071, 25011, 41401, 4019]","[41071, 25011, 41401, 4019]","[41071, 25011, 41401, 4019]","[41071, 25011, 41401, 4019]"


In [65]:
print(merge_diag.shape)
print(merge_diag.SUBJECT_ID.nunique())
merge_diag.HADM_ID.nunique()

(58976, 32)
46520


58976

In [66]:
merge_drug = pd.merge(merge_diag, grouped_drugs, on=['SUBJECT_ID', 'HADM_ID'], how='left')
merge_drug

,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,...,CCS_CODES_diagnosis,CCS_description_diagnosis,LEVE3 CODES,threshold_0.95_diagnosis,threshold_0.88_diagnosis,threshold_0.98_diagnosis,threshold_0.999_diagnosis,DRUG,ATC4,ATC3
0,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,None,UNOBTAINABLE,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,0,...,"[242, 241, 662, 662, 95, 657, 98]","[Poisn ot med, Poison psych, Suicide and inten...","[967, 969, E95, E95, 348, 296, 401]","[-1, 9693, -1, E9503, 3488, 29620, 4019]","[-1, -1, -1, E9503, 3488, 29620, 4019]","[9678, 9693, E9502, E9503, 3488, 29620, 4019]","[9678, 9693, E9502, E9503, 3488, 29620, 4019]",NaN,NaN,NaN
1,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,None,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,...,"[101, 101, 96, 101, 53, 98, 164, 94]","[Coron athero, Coron athero, Hrt valve dx, Cor...","[414, 411, 424, V45, 272, 401, 600, 389]","[41401, 4111, 4241, V4582, 2724, 4019, 60000, ...","[41401, 4111, 4241, V4582, 2724, 4019, 60000, ...","[41401, 4111, 4241, V4582, 2724, 4019, 60000, ...","[41401, 4111, 4241, V4582, 2724, 4019, 60000, ...","[Potassium Chloride, Potassium Chloride, Docus...","[A12BA, A12BA, A06AA, A06AA, A02BA, A02BA, B05...","[A12B, A12B, A06A, A06A, A02B, A02B, B05C, C07..."
2,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,BRAIN MASS,0,...,"[47, 95, 83, 96, 98, 53, 53, 101, 259, 663]","[Ot bnign neo, Oth nerv dx, Epilepsy/cnv, Hrt ...","[225, 348, 780, 424, 401, 272, 272, V45, V45, ...","[2252, 3485, 78039, 4241, 4019, 2720, 2724, V4...","[2252, 3485, 78039, 4241, 4019, 2720, 2724, V4...","[2252, 3485, 78039, 4241, 4019, 2720, 2724, V4...","[2252, 3485, 78039, 4241, 4019, 2720, 2724, V4...","[Heparin, 5% Dextrose, 0.9% Sodium Chloride, 0...","[B01AB, B05CX, A12CA, A12CA, A12CA, A06AB, A02...","[B01A, B05C, A12C, A12C, A12C, A06A, A02B, N02..."
3,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,None,PROTESTANT QUAKER,SINGLE,WHITE,INTERIOR MYOCARDIAL INFARCTION,0,...,"[100, 101, 138, 49]","[Acute MI, Coron athero, Esophgeal dx, DiabMel...","[410, 414, 530, 250]","[41041, 41401, 53081, 25000]","[41041, 41401, 53081, 25000]","[41041, 41401, 53081, 25000]","[41041, 41401, 53081, 25000]","[Potassium Chloride, Heparin, Docusate Sodium,...","[A12BA, B01AB, A06AA, A06AB, A12CA, C07AB, A02...","[A12B, B01A, A06A, A06A, A12C, C07A, A02B, N02..."
4,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,None,UNOBTAINABLE,MARRIED,WHITE,ACUTE CORONARY SYNDROME,0,...,"[100, 50, 101, 98]","[Acute MI, DiabMel w/cm, Coron athero, HTN]","[410, 250, 414, 401]","[41071, 25011, 41401, 4019]","[41071, 25011, 41401, 4019]","[41071, 25011, 41401, 4019]","[41071, 25011, 41401, 4019]","[Potassium Chloride, Potassium Chloride, Potas...","[A12BA, A12BA, A12BA, B01AB, B05CX, B01AB, B01...","[A12B, A12B, A12B, B01A, B05C, B01A, B01A, A12..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,NOT SPECIFIED,SINGLE,WHITE,TRAUMA,0,...,"[228, 60, 235, 235, 235, 235, 172, 2607]","[Fx skull fac, Acut p-h anm, Opn wnd head, Opn...","[802, 285, 870, 873, 873, 873, 620, E81]","[8028, 2851, -1, -1, -1, 87343, 6202, E8161]","[8028, 2851, -1, -1, -1, 87343, -1, -1]","[8028, 2851, -1, 87341, -1, 87343, 6202, E8161]","[8028, 2851, 8701, 87341, 87352, 87343, 6202, ...","[Heparin, Clindamycin, 0.9% Sodium Chloride, 5...","[B01AB, D10AF, A12CA, B05CX, A12CA, A04AA, N02...","[B01A, D10A, A12C, B05C, A12C, A04A, N02A, A12..."
58972,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,SAH,1,...,"[109, 98, 101, 257]","[Acute CVD, HTN, Coron athero, Ot aftercare]","[430, 401, 414, V66]","[430, 4019, 41401, V667]","[430, 4019, 41401, V667]","[430, 4019, 41401, V667]","[430, 4019, 41401, V667]",[Scopolamine Patch]

In [67]:
print(merge_drug.shape)
print(merge_drug.SUBJECT_ID.nunique())
merge_drug.HADM_ID.nunique()

(58976, 35)
46520


58976

In [69]:
merge_drug1 = pd.merge(merge_drug, grouped_drugs1, on=['SUBJECT_ID', 'HADM_ID'], how='left')
merge_drug1

,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,...,threshold_0.98_diagnosis,threshold_0.999_diagnosis,DRUG_x,ATC4,ATC3,DRUG_y,threshold_0.88,threshold_0.95,threshold_0.98,threshold_0.999
0,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,None,UNOBTAINABLE,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,0,...,"[9678, 9693, E9502, E9503, 3488, 29620, 4019]","[9678, 9693, E9502, E9503, 3488, 29620, 4019]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,None,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,...,"[41401, 4111, 4241, V4582, 2724, 4019, 60000, ...","[41401, 4111, 4241, V4582, 2724, 4019, 60000, ...","[Potassium Chloride, Potassium Chloride, Docus...","[A12BA, A12BA, A06AA, A06AA, A02BA, A02BA, B05...","[A12B, A12B, A06A, A06A, A02B, A02B, B05C, C07...","[Syringe, Phenylephrine HCl, Nitroglycerin, LR...","[Syringe, Phenylephrine HCl, Nitroglycerin, LR...","[Syringe, Phenylephrine HCl, Nitroglycerin, LR...","[Syringe, Phenylephrine HCl, Nitroglycerin, LR...","[Syringe, Phenylephrine HCl, Nitroglycerin, LR..."
2,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,BRAIN MASS,0,...,"[2252, 3485, 78039, 4241, 4019, 2720, 2724, V4...","[2252, 3485, 78039, 4241, 4019, 2720, 2724, V4...","[Heparin, 5% Dextrose, 0.9% Sodium Chloride, 0...","[B01AB, B05CX, A12CA, A12CA, A12CA, A06AB, A02...","[B01A, B05C, A12C, A12C, A12C, A06A, A02B, N02...","[SW, 0.9% Sodium Chloride, Dexamethasone, Vanc...","[SW, 0.9% Sodium Chloride, Dexamethasone, Vanc...","[SW, 0.9% Sodium Chloride, Dexamethasone, Vanc...","[SW, 0.9% Sodium Chloride, Dexamethasone, Vanc...","[SW, 0.9% Sodium Chloride, Dexamethasone, Vanc..."
3,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,None,PROTESTANT QUAKER,SINGLE,WHITE,INTERIOR MYOCARDIAL INFARCTION,0,...,"[41041, 41401, 53081, 25000]","[41041, 41401, 53081, 25000]","[Potassium Chloride, Heparin, Docusate Sodium,...","[A12BA, B01AB, A06AA, A06AB, A12CA, C07AB, A02...","[A12B, B01A, A06A, A06A, A12C, C07A, A02B, N02...","[Fentanyl Citrate, Captopril, NS, 1/2 NS, Atro...","[Fentanyl Citrate, Captopril, NS, 1/2 NS, Atro...","[Fentanyl Citrate, Captopril, NS, 1/2 NS, Atro...","[Fentanyl Citrate, Captopril, NS, 1/2 NS, Atro...","[Fentanyl Citrate, Captopril, NS, 1/2 NS, Atro..."
4,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,None,UNOBTAINABLE,MARRIED,WHITE,ACUTE CORONARY SYNDROME,0,...,"[41071, 25011, 41401, 4019]","[41071, 25011, 41401, 4019]","[Potassium Chloride, Potassium Chloride, Potas...","[A12BA, A12BA, A12BA, B01AB, B05CX, B01AB, B01...","[A12B, A12B, A12B, B01A, B05C, B01A, B01A, A12...","[Heparin Sodium, D5 1/2NS, Dextrose 5%, D5W, N...","[Heparin Sodium, D5 1/2NS, Dextrose 5%, D5W, N...","[Heparin Sodium, D5 1/2NS, Dextrose 5%, D5W, N...","[Heparin Sodium, D5 1/2NS, Dextrose 5%, D5W, N...","[Heparin Sodium, D5 1/2NS, Dextrose 5%, D5W, N..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,NOT SPECIFIED,SINGLE,WHITE,TRAUMA,0,...,"[8028, 2851, -1, 87341, -1, 87343, 6202, E8161]","[8028, 2851, 8701, 87341, 87352, 87343, 6202, ...","[Heparin, Clindamycin, 0.9% Sodium Chloride, 5...","[B01AB, D10AF, A12CA, B05CX, A12CA, A04AA, N02...","[B01A, D10A, A12C, B05C, A12C, A04A, N02A, A12...","[Lactated Ringers, Sodium Chloride 0.9% Flush...","[Lactated Ringers, Sodium Chloride 0.9% Flush...","[Lactated Ringers, Sodium Chloride 0.9% Flush...","[Lactated Ringers, Sodium Chloride 0.9% Flush...","[Lactated Ringers, Sodium Chloride 0.9% Flush..."
58972,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,SAH,1,...,"[430, 4019, 41401, V667]","[430, 4019, 41401, V667]",[Scopolamine Patch],[A04AD],[A04A],"[Morphine Sulfate, Sodium Chloride 0.9% Flush...","[Morphine Sulfate, Sodium Chloride 0.9%

In [70]:
print(merge_drug1.shape)
print(merge_drug1.SUBJECT_ID.nunique())
merge_drug1.HADM_ID.nunique()

(58976, 40)
46520


58976

In [72]:
pd.set_option('display.max_columns', None)

merge_drug1.head()

,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,GENDER,EXPIRE_FLAG,DOB,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,LOSRD,age,year_age,ICD9_CODE_procedures,CCS CATEGORY DESCRIPTION_proc,CCS CODES_proc,cat_threshold .95 most frequent_proc,ICD9_CODE_diagnosis,CCS_CODES_diagnosis,CCS_description_diagnosis,LEVE3 CODES,threshold_0.95_diagnosis,threshold_0.88_diagnosis,threshold_0.98_diagnosis,threshold_0.999_diagnosis,DRUG_x,ATC4,ATC3,DRUG_y,threshold_0.88,threshold_0.95,threshold_0.98,threshold_0.999
0,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,None,UNOBTAINABLE,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,0,F,0,2131-05-07,22,165315,2196-04-09,2196-04-10 15:54:00,1,"23714 days, 0:00:00",64.969863,"[9671, 9604, 9634]","['Mech ventil ', 'Mech ventil ', 'nO...","['216', '216', '93']","[9671, 9604, 9634]","[9678, 9693, E9502, E9503, 3488, 29620, 4019]","[242, 241, 662, 662, 95, 657, 98]","[Poisn ot med, Poison psych, Suicide and inten...","[967, 969, E95, E95, 348, 296, 401]","[-1, 9693, -1, E9503, 3488, 29620, 4019]","[-1, -1, -1, E9503, 3488, 29620, 4019]","[9678, 9693, E9502, E9503, 3488, 29620, 4019]","[9678, 9693, E9502, E9503, 3488, 29620, 4019]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,None,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,M,0,2082-07-17,23,152223,2153-09-03,2153-09-08 19:10:00,5,"25980 days, 0:00:00",71.178082,"[3613, 3615, 3961, 8872, 9904, 9907, 9905]","['CABG ', 'CABG ', 'Ex...","['44', '44', '50', '193', '222', '222', '222']","[3613, 3615, 3961, 8872, 9904, 9907, 9905]","[41401, 4111, 4241, V4582, 2724, 4019, 60000, ...","[101, 101, 96, 101, 53, 98, 164, 94]","[Coron athero, Coron athero, Hrt valve dx, Cor...","[414, 411, 424, V45, 272, 401, 600, 389]","[41401, 4111, 4241, V4582, 2724, 4019, 60000, ...","[41401, 4111, 4241, V4582, 2724, 4019, 60000, ...","[41401, 4111, 4241, V4582, 2724, 4019, 60000, ...","[41401, 4111, 4241, V4582, 2724, 4019, 60000, ...","[Potassium Chloride, Potassium Chloride, Docus...","[A12BA, A12BA, A06AA, A06AA, A02BA, A02BA, B05...","[A12B, A12B, A06A, A06A, A02B, A02B, B05C, C07...","[Syringe, Phenylephrine HCl, Nitroglycerin, LR...","[Syringe, Phenylephrine HCl, Nitroglycerin, LR...","[Syringe, Phenylephrine HCl, Nitroglycerin, LR...","[Syringe, Phenylephrine HCl, Nitroglycerin, LR...","[Syringe, Phenylephrine HCl, Nitroglycerin, LR..."
2,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,BRAIN MASS,0,M,0,2082-07-17,23,124321,2157-10-18,2157-10-25 14:00:00,6,"27486 days, 0:00:00",75.304110,[151],['Inc/exc CNS '],['1'],[151],"[2252, 3485, 78039, 4241, 4019, 2720, 2724, V4...","[47, 95, 83, 96, 98, 53, 53, 101, 259, 663]","[Ot bnign neo, Oth nerv dx, Epilepsy/cnv, Hrt ...","[225, 348, 780, 424, 401, 272, 272, V45, V45, ...","[2252, 3485, 78039, 4241, 4019, 2720, 2724, V4...","[2252, 3485, 78039, 4241, 4019, 2720, 2724, V4...","[2252, 3485, 78039, 4241, 4019, 2720, 2724, V4...","[2252, 3485, 78039, 4241, 4019, 2720, 2724, V4...","[Heparin, 5% Dextrose, 0.9% Sodium Chloride, 0...","[B01AB, B05CX, A12CA, A12CA, A12CA, A06AB, A02...","[B01A, B05C, A12C, A12C, A12C, A06A, A02B, N02...","[SW, 0.9% Sodium Chloride, Dexamethasone, Vanc...","[SW, 0.9% Sodium Chloride, Dexamethasone, Vanc...","[SW, 0.9% Sodium Chloride, Dexamethasone, Vanc...","[SW, 0.9% Sodium Chloride, Dexamethasone, Vanc...","[SW, 0.9% Sodium Chloride, Dexamethasone, Vanc..."
3,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,None,PROTESTANT QUAKER,SINGLE,WHITE,INTERIOR MYOCARDIAL INFARCTION,0,M,0,2100-05-31,24,161859,2139-06-06,2139-06-09 12:48:00,2,"14250 days, 0:00:00",39.041096,"[3601, 3723, 8856, 9920, 8852, 3607]","['PTCA ', 'Cardiac cath ', 'Ca...","['45', '47', '47', '231', '47', '63']","[3601, 3723, 8856, 9920, 8852, 3607]","[41041, 41401, 53081, 25000]","[100, 101, 138, 49]","[Acute MI, Coron ather

In [73]:
merge_drug1.to_csv("data/data_preprocess_nonfilteres.csv")

In [38]:
# este es el preprocess que solo considera  procedures
merge_proc.to_csv("data_preprocess_non_filtered.csv")

# Analisis of icd_9 == 9

In [152]:
The code `s_ids = pd.read_csv("ids_9.csv")["SUBJECT_ID"]` is reading a CSV file named "ids_9.csv" and extracting the values from the "SUBJECT_ID" column. It assigns these values to the variable `s_ids`.
s_ids = pd.read_csv("ids_9.csv")["SUBJECT_ID"]
s_hadmids = pd.read_csv("ids_9.csv")["HADM_ID"]

In [156]:
s_ids

0        6398
1       22432
2       22446
3        7676
4       16881
        ...  
3310     5362
3311    14975
3312     8463
3313     5362
3314     8463
Name: SUBJECT_ID, Length: 3315, dtype: int64

In [154]:

desired_ids = s_ids
condition = merge_drug['SUBJECT_ID'].isin(desired_ids)

# Apply the condition to filter the DataFrame
filtered_ds = merge_drug[condition]


In [155]:
filtered_ds

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,HOSPITAL_EXPIRE_FLAG,ICD9_CODE,CCS CATEGORY DESCRIPTION_proc,CCS CODES_proc,cat_threshold .95 most frequent_proc,CCS CODES_diag,LEVE3 CODES,icd9_category,cat_threshold .75_diag,CCS CATEGORY DESCRIPTION_diag,DRUG,ATC4,ATC3
4,36,182104,2131-04-30 07:15:00,2131-05-08 14:00:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,ENGL,NOT SPECIFIED,MARRIED,0,"[3615, 3961, 8856, 8853, 3722, 8872, 3614, 8845]","['CABG ', 'Extrcor circ ', 'Ca...","['44', '50', '47', '47', '47', '193', '44', '1...","[3615, 3961, 8856, 8853, 3722, 8872, 3614, 8845]","[101, 101, 127, 98, 663, 138, 164, 32, 162]","[414, 411, 496, 401, 305, 530, 600, V10, 596]","[Diseases of the Circulatory System, Diseases ...","[41401, 4111, 496, 4019, 3051, 53081, Otro, V1...","['Coron athero', 'Coron athero', 'COPD', 'HTN'...","[Potassium Chloride, Heparin, Docusate Sodium,...","[A12BA, B01AB, A06AA, A06AA, A06AA, A06AB, A02...","[A12B, B01A, A06A, A06A, A06A, A06A, A02B, A02..."
5,36,122659,2131-05-12 19:49:00,2131-05-25 13:30:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,ENGL,NOT SPECIFIED,MARRIED,0,"[3893, 3479, 7761, 7761, 8674, 3403, 3932, 990...","['Ot vasc cath ', 'OR respirato ', 'Ex...","['54', '42', '142', '142', '172', '42', '61', ...","[3893, 3479, 7761, 7761, 8674, 3403, 3932, 990...","[238, 238, 238, 118, 237, 127, 101, 101, 98, 1...","[998, 998, 415, 453, 996, 496, 414, V45, 401, ...","[Injury and Poisoning, Injury and Poisoning, D...","[99831, 99811, Otro, Otro, 99672, 496, 41401, ...","['Complic proc', 'Complic proc', 'Complic proc...","[Potassium Chloride, Potassium Chloride, Potas...","[A12BA, A12BA, A12BA, A12BA, B01AB, B01AB, A06...","[A12B, A12B, A12B, A12B, B01A, B01A, A06A, A06..."
6,36,165660,2134-05-10 11:30:00,2134-05-20 13:16:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,LONG TERM CARE HOSPITAL,Medicare,ENGL,NOT SPECIFIED,MARRIED,0,"[9671, 9604, 3479, 544, 5361, 7861, 9977]","['Mech ventil ', 'Mech ventil ', 'OR...","['216', '216', '42', '99', '86', '148', '231']","[9671, 9604, 3479, 544, 5361, Otro, 9977]","[143, 238, 131, 122, 238, 130, 101, 127, 98, 1...","[553, 415, 518, 486, 997, 518, 414, 496, 401, ...","[Diseases of the Digestive System, Diseases of...","[Otro, Otro, 5185, 486, 99739, 5180, 41400, 49...","['Abdom hernia', 'Complic proc', 'Adlt resp fl...","[Heparin, Heparin, Heparin, Heparin, Heparin, ...","[B01AB, B01AB, B01AB, B01AB, B01AB, C03CA, C03...","[B01A, B01A, B01A, B01A, B01A, C03C, C03C, C03..."
16,368,105889,2137-07-11 17:56:00,2137-07-16 16:00:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,NOT SPECIFIED,WIDOWED,0,NaN,NaN,NaN,NaN,"[122, 127, 100, 157, 158, 117, 59, 138, 651]","[486, 491, 410, 584, 585, 458, 280, 530, 300]","[Diseases of the Respiratory System, Diseases ...","[486, 49121, 41071, 5849, Otro, 4589, 2809, 53...","['Pneumonia', 'COPD', 'Acute MI', 'Ac renl fai...","[Prednisone, Prednisone, Acetaminophen, Hepari...","[A07EA, A07EA, N02BE, B01AB, B01AB, B01AB, A06...","[A07E, A07E, N02B, B01A, B01A, B01A, A06A, A12..."
17,368,138061,2139-12-16 19:48:00,2139-12-22 16:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,NOT SPECIFIED,WIDOWED,0,[9904],['Blood transf '],['222'],[9904],"[100, 122, 131, 108, 55, 99, 55, 127, 101, 100...","[410, 486, 518, 428, 276, 403, 276, 491, 414, ...","[Diseases of the Circulatory System, Diseases ...","[41071, 486, 51882, 4280, 2762, 40391, Otro, O...","['Acute MI', 'Pneumonia', 'Adlt resp fl', 'chf...","[Potassium Chloride, Prednisone, Prednisone, P...","[A12BA, A07EA, A07EA, A07EA, B01AB, A06AA, A06...","[A12B, A07E, A07E, A07E, B01A, A06A, A06A, A12..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19983,98744,184410,2127-06-30 20:19:00,2127-07-04 14:35:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,ENGL,NOT SPE